Load Libraries

In [28]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt
import pickle as pkl

#Classification with KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Load Data and give them a name

In [29]:
def loadDataFrames(name, pathAcc, pathGyr, pathOri):
    df_Acc= pd.read_csv(pathAcc)
    df_Gyr= pd.read_csv(pathGyr)
    df_Ori= pd.read_csv(pathOri)
    print("Data Frames for " + name + " loaded.")
    return df_Acc, df_Gyr, df_Ori	    

#load mixed data
df_mixed_Acc, df_mixed_Gyr, df_mixed_Ori =loadDataFrames('mixed','data/MixedData/Accelerometer.csv', 'data/MixedData/Gyroscope.csv', 'data/MixedData/Orientation.csv')

#load data frames for walking
df_walk_Acc, df_walk_Gyr, df_walk_Ori =loadDataFrames('walking','data/NormalWalk/Accelerometer.csv', 'data/NormalWalk/Gyroscope.csv', 'data/NormalWalk/Orientation.csv')

#load data frames for push ups
df_push_Acc, df_push_Gyr, df_push_Ori =loadDataFrames('push ups','data/PushUps/Accelerometer.csv', 'data/PushUps/Gyroscope.csv', 'data/PushUps/Orientation.csv')

#load data frames for jumping jacks
df_JJ_Acc, df_JJ_Gyr, df_JJ_Ori =loadDataFrames('jumping jacks','data/JJ_rightHand/Accelerometer.csv', 'data/JJ_rightHand/Gyroscope.csv', 'data/JJ_rightHand/Orientation.csv')

#load data JJ2
df_JJ2_Acc, df_JJ2_Gyr, df_JJ2_Ori =loadDataFrames('jumping jacks 2','data/JJ2/Accelerometer.csv', 'data/JJ2/Gyroscope.csv', 'data/JJ2/Orientation.csv')

#load data pushups2
df_push2_Acc, df_push2_Gyr, df_push2_Ori =loadDataFrames('push ups 2','data/PushUps2/Accelerometer.csv', 'data/PushUps2/Gyroscope.csv', 'data/PushUps2/Orientation.csv')

#load data walk2
df_walk2_Acc, df_walk2_Gyr, df_walk2_Ori =loadDataFrames('walking 2','data/Walk2/Accelerometer.csv', 'data/Walk2/Gyroscope.csv', 'data/Walk2/Orientation.csv')


Data Frames for mixed loaded.
Data Frames for walking loaded.
Data Frames for push ups loaded.
Data Frames for jumping jacks loaded.
Data Frames for jumping jacks 2 loaded.
Data Frames for push ups 2 loaded.
Data Frames for walking 2 loaded.


In [ ]:
df_push_Acc.head(10)
#ggf. spalten umbennen zb. z zu z_Acc etc. 
#ggf. Spalten dropen die ich nicht brauche wie yawn, pitch, roll oder so, mal schauen wie die Daten aussehen

In [ ]:
df_push_Gyr.plot(x='time', y='z', kind='line')
df_push_Gyr.plot(x='time', y='y', kind='line')
df_push_Gyr.plot(x='time', y='x', kind='line')

Define Functions

In [30]:
#Function to plot Gyr Data
def plotDataGyr(data, title):
    plt.plot(data['time'], data['z'], color='red', marker='.')
    plt.plot(data['time'], data['y'], color='blue', marker='.')
    plt.plot(data['time'], data['x'], color='green', marker='.')
    plt.xlabel('time')
    plt.ylabel('x,y,z')
    plt.legend(['z', 'y', 'x'])
    plt.title(title)
    plt.show()

#Function to Plot Acc Data
def plotDataAcc(data, title):
    plt.plot(data['time'], data['z'], color='red', marker='.')
    plt.plot(data['time'], data['y'], color='blue', marker='.')
    plt.plot(data['time'], data['x'], color='green', marker='.')
    plt.xlabel('time')
    plt.ylabel('x,y,z')
    plt.legend(['z', 'y', 'x'])
    plt.title(title)
    plt.show()

#Function to Plot Ori Data
def plotDataOri(data, title):
    plt.plot(data['time'], data['qz'], color='red', marker='.')
    plt.plot(data['time'], data['qy'], color='blue', marker='.')
    plt.plot(data['time'], data['qx'], color='green', marker='.')
    plt.plot(data['time'], data['qw'], color='yellow', marker='.')
    #plt.plot(data['time'], data['roll'], color='black', marker='.')
    plt.xlabel('time')
    plt.ylabel('qz,qy,qx,qw,roll')
    plt.legend(['qz', 'qy', 'qx', 'qw', 'roll'])
    plt.title(title)
    plt.show()

#Concat Funktion
def concatDataFrames(name, df_Acc, df_Gyr, df_Ori):
  df_Gyr.drop(['time','seconds_elapsed'], axis=1, inplace=True)
  df_Ori.drop(['time','seconds_elapsed'], axis=1, inplace=True)
  df = pd.concat([df_Acc, df_Gyr, df_Ori], axis=1)
  print("Data Frames for " + name + " concatenated.")
  return df

In [ ]:
plotDataGyr(df_walk2_Gyr, 'Gyr Data for walking 2')

In [ ]:
#Plot Walk
plotDataGyr(df_walk_Gyr, 'Gyroscope Walk')

#Plot PushUps
plotDataGyr(df_push_Gyr, 'Gyroscope PushUps')

#Plot JumpingJacks
plotDataGyr(df_JJ_Gyr, 'Gyroscope JumpingJacks')

In [ ]:
#Plotting Accelerometer Data

#Plot Walk
plotDataAcc(df_walk_Acc, 'Accelerometer Walk')

#Plot PushUps
plotDataAcc(df_push_Acc, 'Accelerometer PushUps')

#Plot JumpingJacks
plotDataAcc(df_JJ_Acc, 'Accelerometer JumpingJacks')

In [ ]:
#Plot Walk
plotDataOri(df_walk_Ori, 'Orientation Walk')

#Plot PushUps
plotDataOri(df_push_Ori, 'Orientation PushUps')

#Plot JumpingJacks
plotDataOri(df_JJ_Ori, 'Orientation JumpingJacks')
    

Concat Dataframes

In [31]:
#Concat DataFrames Walk
df_walk = concatDataFrames('walking', df_walk_Acc, df_walk_Gyr, df_walk_Ori)

#Concat DataFrames PushUps
df_push = concatDataFrames('push ups', df_push_Acc, df_push_Gyr, df_push_Ori)

#Concat DataFrames JumpingJacks
df_JJ = concatDataFrames('jumping jacks', df_JJ_Acc, df_JJ_Gyr, df_JJ_Ori)

#Concat DataFrames JumpingJacks2
df_JJ2 = concatDataFrames('jumping jacks 2', df_JJ2_Acc, df_JJ2_Gyr, df_JJ2_Ori)

#Concat DataFrames PushUps2
df_push2 = concatDataFrames('push ups 2', df_push2_Acc, df_push2_Gyr, df_push2_Ori)

#Concat DataFrames Walk2
df_walk2 = concatDataFrames('walking 2', df_walk2_Acc, df_walk2_Gyr, df_walk2_Ori)

#Concat Mixed DataFrames
df_mixed = concatDataFrames('mixed', df_mixed_Acc, df_mixed_Gyr, df_mixed_Ori)
df_mixed.head(10)

Data Frames for walking concatenated.
Data Frames for push ups concatenated.
Data Frames for jumping jacks concatenated.
Data Frames for jumping jacks 2 concatenated.
Data Frames for push ups 2 concatenated.
Data Frames for walking 2 concatenated.
Data Frames for mixed concatenated.


,time,seconds_elapsed,z,y,x,z,y,x,qz,qy,qx,qw,roll,pitch,yaw
0,1683885692643028200,0.177028,-2.262210,0.806162,-0.278753,1.603674,1.674687,1.103528,-0.811517,0.141371,0.161673,0.543430,0.429760,0.053760,1.949730
1,1683885692659893500,0.193894,-0.917953,0.798062,-0.267378,1.430036,3.169627,1.821600,-0.802009,0.128837,0.182105,0.554094,0.449906,0.004850,1.931321
2,1683885692676774100,0.210774,0.146413,2.362878,0.174195,0.722806,5.473352,2.473699,-0.790269,0.120772,0.199763,0.566555,0.469977,-0.035477,1.906112
3,1683885692693654300,0.227654,-0.701164,4.871797,1.817624,0.776257,3.543477,2.811965,-0.779568,0.117154,0.210182,0.578249,0.482549,-0.060452,1.880066
4,1683885692710510800,0.244511,-2.731052,5.407081,0.866909,1.809077,1.557248,2.462245,-0.774531,0.114960,0.200069,0.588947,0.462370,-0.057611,1.854921
5,1683885692727390000,0.261390,-2.588329,5.020437,0.968598,2.393064,1.090853,1.873524,-0.775383,0.116430,0.173566,0.595903,0.420332,-0.026304,1.836698
6,1683885692744263000,0.278263,-0.897709,4.194711,1.220492,2.288454,1.253648,1.089478,-0.778063,0.121911,0.147691,0.598284,0.385186,0.012987,1.828034
7,1683885692761133800,0.295134,0.574014,4.251270,0.604241,1.736385,-0.034208,0.004429,-0.778929,0.128120,0.134300,0.599014,0.371469,0.038707,1.823189
8,1683885692777996800,0.311997,-0.116035,1.576437,0.299314,0.749226,-1.894904,-1.542740,-0.779003,0.131234,0.131398,0.598888,0.370745,0.047095,1.821925
9,1683885692794875600,0.328876,-0.471364,1.944724,0.045150,-0.203265,-2.515390,-2.413681,-0.779734,0.131699,0.132513,0.597588,0.373048,0.047021,1.824888


In [ ]:
df_walk.describe()


**KNN Classifikation**
1. Label Data
2. Concat Dataframes
3. Split Data
4. Train Model
5. Predict
6. Evaluate

1. Label Data

In [65]:
#Classification with KNN
#Function to label the data
def getData(data, label):
    data['label'] = label
    return data

#Label Data

#Label Data Walk
df_walk = getData(df_walk, 'walk')
#df_walk.head(10)

#Label Data PushUps
df_push = getData(df_push, 'pushups')
#df_push.head(10)

#Label Data JumpingJacks
df_JJ = getData(df_JJ, 'jumpingjacks')
#df_JJ.head(10)

#Label Data JumpingJacks2
df_JJ2 = getData(df_JJ2, 'jumpingjacks')

#Label Data PushUps2
df_push2 = getData(df_push2, 'pushups')

#Label Data Walk2
df_walk2 = getData(df_walk2, 'walk')

2. Concat Dataframes

In [91]:
#Concat DataFrames
df = pd.concat([df_walk, df_push, df_JJ], axis=0)
df_test = pd.concat([df_JJ2, df_push2, df_walk2], axis=0)

#Short Df_test for predciton
df_test= df_test.head(1517)



#Fill NaN Values with propagation
df.fillna(method='ffill', inplace=True)
df_test.fillna(method='ffill', inplace=True)



#kein random split sondern spezifizieren
#erste 70% train, letzte 30% test
#fünf trainings df und quasi im train nur die ersten 3 und die zwei anderen dann als test

In [92]:
print('Test: ', len(df_test))
print('Df: ', len(df))

df_test.drop(['label'], axis=1, inplace=True)

Test:  1517
Df:  5055


ToDo: Marvin
#gemischten Datensatz vlt mal testen, weiß noch nicht genau wie ich den einbauen soll


pickle wird benutzt um das trainierte Modell zu speichern und dann in streamlit zu laden, dort können wir es dann auf den datensatz anwenden
pkl.dump(knn, open('knn.pkl', 'wb'))

In [93]:
df_test.head(10)

,time,seconds_elapsed,z,y,x,z,y,x,qz,qy,qx,qw,roll,pitch,yaw
0,1682933646094084600,0.218085,-0.507880,0.094884,-0.380927,-0.349262,-0.716240,0.580627,0.868939,0.254566,0.189236,0.379910,-0.167969,-0.626348,-2.371765
1,1682933646110957600,0.234958,-0.913389,0.015965,0.005873,-0.325897,-0.513279,0.549321,0.865965,0.257832,0.193772,0.382204,-0.173147,-0.636853,-2.367502
2,1682933646127819000,0.251819,-0.139279,0.024436,0.523408,-0.283136,-0.455553,0.505949,0.863324,0.260324,0.197735,0.384454,-0.177756,-0.645412,-2.363265
3,1682933646144703200,0.268703,-0.217505,-0.247191,0.615641,-0.267254,-0.449444,0.540005,0.861486,0.260923,0.200886,0.386531,-0.182354,-0.649589,-2.359635
4,1682933646161566200,0.285566,-0.077660,-0.546501,0.701738,-0.299935,-0.441503,0.584293,0.860433,0.260159,0.203194,0.388182,-0.186651,-0.650332,-2.357066
5,1682933646178443800,0.302444,-0.550319,-0.498499,0.414404,-0.306654,-0.449597,0.547641,0.859936,0.258639,0.205233,0.389224,-0.191541,-0.649249,-2.356166
6,1682933646195317200,0.319317,-0.924318,-0.151186,0.267515,-0.255189,-0.402407,0.363007,0.859326,0.257294,0.208218,0.389876,-0.198690,-0.649210,-2.356797
7,1682933646212204300,0.336204,-0.210531,0.810930,0.246110,-0.175776,-0.391106,0.096822,0.858612,0.256855,0.211411,0.390021,-0.205961,-0.651006,-2.358566
8,1682933646229073400,0.353073,-0.475749,0.320146,0.367136,-0.080787,-0.320093,-0.031307,0.857659,0.257632,0.214681,0.389819,-0.212695,-0.655171,-2.360909
9,1682933646245941200,0.369941,-0.736414,0.243847,0.172139,0.001680,-0.350026,-0.055741,0.856176,0.259675,0.218086,0.389834,-0.218501,-0.661993,-2.362403


In [ ]:
#Show Training Data
def showXTrainingData(data):
  print(data.shape)
  print(data.columns)
  plt.plot(data['z'], color='red', marker='.')#
  plt.plot(data['y'], color='blue', marker='.')#
  plt.plot(data['x'], color='green', marker='.')#
  plt.plot(data['qz'], color='yellow', marker='.')#
  plt.plot(data['qy'], color='black', marker='.')#
  plt.plot(data['qx'], color='orange', marker='.')#
  plt.plot(data['qw'], color='pink', marker='.')#
  plt.plot(data['roll'], color='purple', marker='.')#
  plt.xlabel('time')
  plt.ylabel('x,y,z,qz,qy,qx,qw,roll')
  plt.legend(['z', 'y', 'x', 'qz', 'qy', 'qx', 'qw', 'roll'])
  plt.title('Train Data')
  plt.show()

showXTrainingData(X_train)


In [33]:
#Concat Test Data2
df_test = pd.concat([df_JJ2, df_push2, df_walk2], axis=0)
df_test.fillna(method='ffill', inplace=True)
df_test.drop(['label'], axis=1, inplace=True)

3. Split Data

In [94]:
#Split Data erste 70% train, letzte 30% test
X = df.drop(['label'], axis=1)
y = df['label']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

4. Train Model

In [95]:
#KNN
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(X_train, y_train)

#size passt noch nicht, da der ytest mehr data hat als der ypred
#shuffle data and then split

KNeighborsClassifier(n_neighbors=3, weights='distance')

5. Predict

In [96]:
y_pred = knn.predict(df_test)
print(len(y_test))

1517


6. Evaluate

In [107]:
#Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

#Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(accuracy * 100, '%')


[[  0   0 379]
 [  0   0 611]
 [  0   0 527]]
34.73961766644694 %


In [ ]:
#Split Data Mixed
X_mixed = df_mixed.drop(['label'], axis=1)
y_mixed = df_mixed['label']
X_train_mixed, X_test_mixed, y_train_mixed, y_test_mixed = train_test_split(X_mixed, y_mixed, test_size=0.3)

#KNN
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')
knn.fit(X_train, y_train)
y_pred_mixed = knn.predict(X_test)